## Generate Pizza Ingredient Proportions

In [184]:
import pandas as pd
import numpy as np

# --- Load pizza types dataset ---
# Each row looks like:
# cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno Peppers, Fontina Cheese, Gouda Cheese"

def parse_ingredients(s):
    # Handles quoted comma-separated ingredients
    if pd.isna(s):
        return []
    return [i.strip() for i in s.split(",")]

pizza_types = pd.read_csv("data/pizza_types.csv", converters={"ingredients": parse_ingredients})


# --- Define base ingredient units (for display) ---
ingredient_units = {
    "Tomatoes": "kg",
    "Red Peppers": "kg",
    "Green Peppers": "kg",
    "Red Onions": "kg",
    "Mushrooms": "kg",
    "Spinach": "kg",
    "Corn": "kg",
    "Garlic": "kg",
    "Jalapeno Peppers": "kg",
    "Pineapple": "kg",
    "Artichokes": "kg",
    "Zucchini": "kg",
    "Eggplant": "kg",
    "Bacon": "packet",
    "Chicken": "packet",
    "Pepperoni": "packet",
    "Sausage": "packet",
    "Mozzarella Cheese": "packet",
    "Feta Cheese": "packet",
    "Fontina Cheese": "packet",
    "Gouda Cheese": "packet",
    "Asiago Cheese": "packet",
    "Goat Cheese": "packet",
    "Provolone Cheese": "packet",
    "Ricotta Cheese": "packet",
    "Blue Cheese": "packet",
    "Barbecue Sauce": "bucket",
    "Pesto Sauce": "bucket",
    "Tomato Sauce": "bucket",
    "Alfredo Sauce": "bucket",
    "Chipotle Sauce": "bucket",
    "Thai Sweet Chilli Sauce": "bucket"
}


# --- Default sauce & cheese rule ---
def ensure_base_ingredients(ingredients):
    has_sauce = any("Sauce" in i for i in ingredients)
    has_cheese = any("Cheese" in i for i in ingredients)
    if not has_sauce:
        ingredients.append("Tomato Sauce")
    if not has_cheese:
        ingredients.append("Mozzarella Cheese")
    return ingredients


# --- Expand dataset ---
rows = []
for _, row in pizza_types.iterrows():
    ingredients = ensure_base_ingredients(row["ingredients"])

    for ing in ingredients:
        unit = ingredient_units.get(ing, "unit")

        # Random quantity of ingredient used (between 0.1 and 0.7)
        quantity = round(np.random.uniform(0.1, 0.7), 2)

        rows.append({
            "pizza_type_id": row["pizza_type_id"],
            "pizza_name": row["name"],
            "category": row["category"],
            "ingredient": ing,
            "quantity_unit": unit,
            "fraction_used": quantity
        })


# --- Create final augmented dataset ---
augmented_df = pd.DataFrame(rows)

# Normalize fractions per pizza to sum exactly to 1
augmented_df["fraction_used"] = (
    augmented_df.groupby("pizza_type_id")["fraction_used"]
    .transform(lambda x: x / x.sum())
    .round(2)
)

print(augmented_df.head(10))


  pizza_type_id                    pizza_name category         ingredient  \
0       bbq_ckn    The Barbecue Chicken Pizza  Chicken  Barbecued Chicken   
1       bbq_ckn    The Barbecue Chicken Pizza  Chicken        Red Peppers   
2       bbq_ckn    The Barbecue Chicken Pizza  Chicken      Green Peppers   
3       bbq_ckn    The Barbecue Chicken Pizza  Chicken           Tomatoes   
4       bbq_ckn    The Barbecue Chicken Pizza  Chicken         Red Onions   
5       bbq_ckn    The Barbecue Chicken Pizza  Chicken     Barbecue Sauce   
6       bbq_ckn    The Barbecue Chicken Pizza  Chicken  Mozzarella Cheese   
7      cali_ckn  The California Chicken Pizza  Chicken            Chicken   
8      cali_ckn  The California Chicken Pizza  Chicken          Artichoke   
9      cali_ckn  The California Chicken Pizza  Chicken            Spinach   

  quantity_unit  fraction_used  
0          unit           0.21  
1            kg           0.15  
2            kg           0.16  
3            kg     

## List Ingredient Quantities as Matrix

In [185]:
# --- Pivot to wide (sparse) format ---
ingredient_matrix = augmented_df.pivot_table(
    index=["pizza_type_id", "pizza_name", "category"],
    columns="ingredient",
    values="fraction_used",
    fill_value=0
)

ingredient_matrix = ingredient_matrix.astype("Sparse[float]")

# --- Save to CSV ---
ingredient_matrix.to_csv("data/pizza_ingredient_matrix.csv")

print("✅ Sparse pizza ingredient matrix saved as: data/pizza_ingredient_matrix.csv")
print(ingredient_matrix.head())


✅ Sparse pizza ingredient matrix saved as: data/pizza_ingredient_matrix.csv
ingredient                                           Alfredo Sauce  Anchovies  \
pizza_type_id pizza_name                   category                             
bbq_ckn       The Barbecue Chicken Pizza   Chicken             0.0        0.0   
big_meat      The Big Meat Pizza           Classic             0.0        0.0   
brie_carre    The Brie Carre Pizza         Supreme             0.0        0.0   
calabrese     The Calabrese Pizza          Supreme             0.0        0.0   
cali_ckn      The California Chicken Pizza Chicken             0.0        0.0   

ingredient                                           Artichoke  Artichokes  \
pizza_type_id pizza_name                   category                          
bbq_ckn       The Barbecue Chicken Pizza   Chicken         0.0         0.0   
big_meat      The Big Meat Pizza           Classic         0.0         0.0   
brie_carre    The Brie Carre Pizza         S

## Generate Price per Kg for Ingredients

In [ ]:
import pandas as pd
import numpy as np

# --- Load ingredient matrix ---
ingredient_matrix = pd.read_csv("data/pizza_ingredient_matrix.csv", index_col=[0,1,2])

# --- Ingredient prices ($/kg) ---
ingredient_prices = {
    "Alfredo Sauce": 6.00,
    "Anchovies": 18.00,
    "Artichoke": 6.50,
    "Artichokes": 6.50,
    "Arugula": 10.00,
    "Asiago Cheese": 20.00,
    "Bacon": 12.00,
    "Barbecue Sauce": 3.50,
    "Barbecued Chicken": 9.00,
    "Beef Chuck Roast": 9.00,
    "Blue Cheese": 16.00,
    "Brie Carre Cheese": 18.00,
    "Calabrese Salami": 15.00,
    "Capocollo": 18.00,
    "Caramelized Onions": 5.00,
    "Chicken": 9.00,
    "Chipotle Sauce": 6.00,
    "Chorizo Sausage": 10.00,
    "Cilantro": 8.00,
    "Coarse Sicilian Salami": 15.00,
    "Corn": 1.80,
    "Eggplant": 2.50,
    "Feta Cheese": 10.00,
    "Fontina Cheese": 12.00,
    "Friggitello Peppers": 6.00,
    "Garlic": 6.00,
    "Genoa Salami": 15.00,
    "Goat Cheese": 18.00,
    "Gorgonzola Piccante Cheese": 18.00,
    "Gouda Cheese": 10.00,
    "Green Olives": 6.50,
    "Green Peppers": 3.50,
    "Italian Sausage": 8.50,
    "Jalapeno Peppers": 3.00,
    "Kalamata Olives": 8.00,
    "Luganega Sausage": 12.00,
    "Mozzarella Cheese": 6.00,
    "Mushrooms": 4.00,
    "Nduja Salami": 14.00,
    "Onions": 1.20,
    "Oregano": 20.00,
    "Pancetta": 14.00,
    "Parmigiano Reggiano Cheese": 18.00,
    "Pears": 2.00,
    "Peperoncini verdi": 6.00,
    "Pepperoni": 10.00,
    "Pesto Sauce": 8.00,
    "Pineapple": 2.50,
    "Plum Tomatoes": 2.50,
    "Prosciutto": 45.00,
    "Prosciutto di San Daniele": 45.00,
    "Provolone Cheese": 9.00,
    "Red Onions": 1.20,
    "Red Peppers": 3.50,
    "Ricotta Cheese": 6.50,
    "Romano Cheese": 14.00,
    "Sliced Ham": 9.00,
    "Smoked Gouda Cheese": 12.00,
    "Soppressata Salami": 15.00,
    "Spinach": 4.00,
    "Sun-dried Tomatoes": 12.00,
    "Thai Sweet Chilli Sauce": 6.00,
    "Thyme": 20.00,
    "Tomato Sauce": 2.50,
    "Tomatoes": 2.50,
    "Zucchini": 2.50
}

# --- Save ingredient prices to CSV ---
pd.DataFrame(list(ingredient_prices.items()), columns=["ingredient", "price_per_kg"]).to_csv(
    "data/ingredient_prices.csv", index=False
)
print("✅ Saved ingredient prices to: data/ingredient_prices.csv")

# --- Compute pizza cost ---
prices = pd.Series(ingredient_prices)
common_cols = [c for c in ingredient_matrix.columns if c in prices.index]
pizza_costs = ingredient_matrix[common_cols].dot(prices[common_cols])

# --- Top 10 most expensive pizzas ---
top10 = pizza_costs.sort_values(ascending=False).head(10)
print("\n🍕 Top 10 Most Expensive Pizzas ($):")
print(top10.round(2))

✅ Saved ingredient prices to: data/ingredient_prices.csv

🍕 Top 10 Most Expensive Pizzas ($):
pizza_type_id  pizza_name                        category
prsc_argla     The Prosciutto and Arugula Pizza  Supreme     11.54
ital_cpcllo    The Italian Capocollo Pizza       Classic     10.74
five_cheese    The Five Cheese Pizza             Veggie      10.67
peppr_salami   The Pepper Salami Pizza           Supreme     10.58
brie_carre     The Brie Carre Pizza              Supreme     10.27
spicy_ital     The Spicy Italian Pizza           Supreme      9.07
soppressata    The Soppressata Pizza             Supreme      8.56
four_cheese    The Four Cheese Pizza             Veggie       8.28
ckn_alfredo    The Chicken Alfredo Pizza         Chicken      7.90
big_meat       The Big Meat Pizza                Classic      7.58
dtype: float64


## Add Time of Day Bucketing for orders

In [190]:
import pandas as pd

# Read the CSV
df = pd.read_csv("data/orders.csv")

# Convert the time column to datetime.time
df["time"] = pd.to_datetime(df["time"], format="%H:%M:%S").dt.time

# Define a function to assign the time bucket
def get_time_bucket(t):
    if t >= pd.to_datetime("09:00:00").time() and t < pd.to_datetime("15:00:00").time():
        return "Lunch"
    elif t >= pd.to_datetime("15:00:00").time() and t < pd.to_datetime("18:00:00").time():
        return "Afternoon"
    elif t >= pd.to_datetime("18:00:00").time() and t < pd.to_datetime("23:59:59").time():
        return "Dinner"
    else:
        return "Other"

# Apply the function
df["time_bucket"] = df["time"].apply(get_time_bucket)

# Save the new CSV
df.to_csv("data/orders_with_buckets.csv", index=False)

print(df)


       order_id        date      time time_bucket
0             1  01/01/2015  11:38:36       Lunch
1             2  01/01/2015  11:57:40       Lunch
2             3  01/01/2015  12:12:28       Lunch
3             4  01/01/2015  12:16:31       Lunch
4             5  01/01/2015  12:21:30       Lunch
...         ...         ...       ...         ...
21345     21346  31/12/2015  20:51:07      Dinner
21346     21347  31/12/2015  21:14:37      Dinner
21347     21348  31/12/2015  21:23:10      Dinner
21348     21349  31/12/2015  22:09:54      Dinner
21349     21350  31/12/2015  23:02:05      Dinner

[21350 rows x 4 columns]
